# **Transcripción de audios** mediante Whisper

- Documento original Notebook creado por [DotCSV](https://www.youtube.com/channel/UCy5znSnfMsDwaLlROnZ7Qbg)
- Editado por Carlos Macheratti

Pensado para ser utilizado desde Google Colab.

### Consideraciones importantes:
1. Para utilizar el servicio es necesario, disponer de acceso a internet y de una cuenta de gmail, con un poco de espacio libre en el drive.
2. Si es la primera vez que usás este notebook hacé una copia en tu drive desde `"Menu" > "Archivo" > "Guardar una copia en Drive"`
3. Controla que esté activada la **Aceleración por hardware** con GPU. Revisar en `"Menú" > "Entorno de ejecución" > "Cambiar tipo de entorno de ejecución"` que esté tildada la opción "GPU T4"
4. Los datos almacenados en el procesamiento se eliminan al cerrar la sesión de Colab. Por lo que cada vez que lo utilices deberás realizar los pasos siguientes.

## 1 - **Instalar** las librería de Whisper y otros requerimientos

In [ ]:
#@title Ejecutar esta celda.

!pip install git+https://github.com/openai/whisper.git
!pip install jiwer
!pip install ffmpeg

from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

## 2 - **Crear** la estructura de carpetas

In [ ]:
#@title Ejecutar esta celda para organizar automáticamente las carpetas del proyecto.
!mkdir -p audio texto
!rm -rf sample_data

## 3 - **Almacenar** los audios

1. Abrir el Navegador de archivos (desde `"Menú" > "Mostrar navegador de archivos"`).
2. Subir tus audios en la carpeta "audio/". Para lo cual se necesita presionar sobre el nombre de la carpeta y elegir desde el menu contextual la opción "Subir".

## 4 - **Transcribir** los audios

In [ ]:
#@title Ejecuta esta celda para realizar la conversión.
!for archivo in ./audio/*;do whisper "./$archivo" --task transcribe --model medium --verbose False --output_dir texto; done

## 5 - **Descargar** las transcripciones

1. Ejecutar la siguiente celda para empaquetar todos los textos transcriptos hacia el archivo **"transcripciones.zip"**, ubicado en la misma carpeta del proyecto. En caso de que no muestre el nuevo archivo actualizar desde el botón con el ícono de "carpeta y flecha cicular".
2. Haciendo click derecho (o manteniendo presionado en dispositivos moviles) sobre ese archivo se habilita un menú con la opción **"Descargar"**.


In [ ]:
#@title Ejecutar esta celda para empaquetar:
!zip -r ./transcripciones.zip ./texto/

## 6 - NOTA sobre las transcripciones.

Los textos están en formato de texto plano. Estructurados para distintas utilidades.

El que nos servirá, principalmente, será el que está en formato "txt".

Dicho formato puede abrirse directamente con cualquier software de edición de texto plano (por ejemplo: Block de notas, Sublime Text) o enriquecido (por ejemplo: Microsoft Word, Google Docs, LibreoOffice Writer).